# Example 1: Setting Up Your Archive

This first example runs through the basics of getting started with `MetaViz`. This includes a run-down on the organization of the code, an introduction to the `config.py` file, and instantiating an `Archive` object that you can begin playing with.

More complex interactions with your `Archive` will be covered in Example 2.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#### Import `MetaViz`

First, let's import the `MetaViz` package. We'll do this in two parts.

In [ ]:
import MetaViz as mv
from MetaViz import Archive